<p align="center">
<b><font size="9" color="#000080">PORTICO PLANO ETABS</font></b>
<div align="center">
    <div style="display: inline-block;">
    <span style="font-size: 18px;position: relative; bottom: 13px;">Richard Cueto Calderon | Update :7/05/204 | API ETBAS | Python</span>
    </div>
</div>
</p>

Vamos a crear un portico de dos paños

Input:
+ L1
+ L2
+ H1
+ Fc
+ Seccion de columnas y vigas

In [1]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
AttachToInstance = False
SpecifyPath = True
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 19\ETABS.exe"
APIPath = r'D:\Users\User(D)\Documentos\GitHub\freeproyect\Etabs\Modelos'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'Portico.edb'
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
if AttachToInstance:
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    myETABSObject.ApplicationStart()
SapModel = myETABSObject.SapModel
SapModel.InitializeNewModel()

0

In [ ]:
AttachToInstance = False
SpecifyPath = True
ProgramPath = "C:\Program Files\Computers and Structures\SAP2000 22\SAP2000.exe"
APIPath = r'D:\Users\User(D)\Documentos\GitHub\freeproyect\SAP2000\Modelos'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'portico.sdb'
helper = comtypes.client.CreateObject('SAP2000v1.Helper')
helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
if AttachToInstance:
    try:
        mySapObject = helper.GetObject("CSI.SAP2000.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            mySapObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    mySapObject.ApplicationStart()
SapModel = mySapObject.SapModel
SapModel.InitializeNewModel()

In [3]:
# Creamos un nuevo modelo en blanco
ret=SapModel.File.NewBlank()

In [4]:
# Cambiamos unidades a ton_m
Tonf_m_C=12
ret = SapModel.SetPresentUnits(Tonf_m_C)

In [5]:
# Definicion de material concreto
fc=280
material_type=2
material_name="fc"+str(fc)+"kg/cm2" #fc=280kg/cm2
ret=SapModel.PropMaterial.SetMaterial(material_name, material_type)
# Definir propiedades mecanicas del material
E=15000*fc**0.5             #kg/cm2
v=0.2
CT=0.0000055
ret=SapModel.PropMaterial.SetMPIsotropic(material_name,E,v,CT)

In [6]:
# Creamos seccion de viga y columna
# Viga
b_v=0.3
h_v=0.6
nombre_viga='v '+str(b_v)+'x'+str(h_v)          #V 0.30x0.60
ret = SapModel.PropFrame.SetRectangle(nombre_viga, material_name, h_v, b_v)

# Columna
b_c=0.45
h_c=0.45
nombre_columna='v '+str(b_c)+'x'+str(h_c)          #V 0.45x0.45
ret = SapModel.PropFrame.SetRectangle(nombre_columna, material_name, h_c, b_c)

In [7]:
# definimos modificaciones
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers(nombre_viga, ModValue)
ret = SapModel.PropFrame.SetModifiers(nombre_columna, ModValue)

**CREACION DE PORTICO**

Esta creacion ser de manera manual y mecanica

+ Creacion de Frame por coordendas
+ Ingresamos paramteros


In [8]:
h=3
L1=4
L2=7

FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
FrameName4 = ' '
FrameName5 = ' '

# columnas
[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, h, FrameName1, nombre_columna, '1', 'Global')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(L1, 0, 0, L1, 0, h, FrameName2,nombre_columna, '2', 'Global')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(L1+L2, 0, 0,L1+L2, 0, h, FrameName3, nombre_columna, '3', 'Global')

# vigas
[FrameName4, ret] = SapModel.FrameObj.AddByCoord(0, 0, h, L1, 0, h, FrameName4, nombre_viga, '4', 'Global')
[FrameName5, ret] = SapModel.FrameObj.AddByCoord(L1, 0, h,L1+L2, 0, h, FrameName5, nombre_viga, '5', 'Global')

In [9]:
#assign point object restraint at base
Pointi = ' '
Pointj = ' '
Restraint = [True, True, True, True, True, True]
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(FrameName1, Pointi, Pointj)
ret = SapModel.PointObj.SetRestraint(Pointi, Restraint)

Pointi = ' '
Pointj = ' '
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(FrameName2, Pointi, Pointj)
ret = SapModel.PointObj.SetRestraint(Pointi, Restraint)

Pointi = ' '
Pointj = ' '
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(FrameName3, Pointi, Pointj)
ret = SapModel.PointObj.SetRestraint(Pointi, Restraint)

In [10]:
# Creamos un LoadPattern
ret = SapModel.LoadPatterns.Add('FH', 8, 0, True)

# Asignar carga distribuida
ret = SapModel.FrameObj.SetLoadDistributed(FrameName4,'Dead',1,2,0,1,-0.4,-0.4,'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName4,'Live',1,2,0,1,-0.25,-0.25,'Local')

ret = SapModel.FrameObj.SetLoadDistributed(FrameName5,'Dead',1,2,0,1,-0.4,-0.4,'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName5,'Live',1,2,0,1,-0.25,-0.25,'Local')
# Asignar carga puntual
PointLoadValue = [1,0,0,0,0,0]
Pointi=""
Pointj=""
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(FrameName1, Pointi, Pointj)
ret = SapModel.PointObj.SetLoadForce(Pointj,'FH',PointLoadValue)

[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(FrameName3, Pointi, Pointj)
ret = SapModel.PointObj.SetLoadForce(Pointj,'FH',PointLoadValue)

In [11]:
#refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

In [12]:
#save model
ret = SapModel.File.Save(ModelPath)

In [13]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()